In [1]:
import pandas as pd

# Define the folder
FOLDER = '../data/processed/'

# Load the data
df_beers = pd.read_parquet(FOLDER + 'beers.pq')
df_breweries = pd.read_parquet(FOLDER + 'breweries.pq')
df_users = pd.read_parquet(FOLDER + 'users.pq')
df_ratings_no_text = pd.read_parquet(FOLDER + 'ratings_no_text.pq')


In [2]:
# Merge user and brewery locations into the ratings data
df_ratings_no_text = df_ratings_no_text.merge(
    df_users[['user_id', 'location']].rename(columns={'location': 'location_user'}),
    on='user_id'
)
df_ratings_no_text = df_ratings_no_text.merge(
    df_breweries[['id', 'location']].rename(columns={'location': 'location_brewery', 'id': 'brewery_id'}),
    on='brewery_id'
)

# Filter for years from 2002 onwards
df_ratings_no_text['date'] = pd.to_datetime(df_ratings_no_text['date'])
df_ratings_no_text = df_ratings_no_text[df_ratings_no_text['date'].dt.year >= 2002]


In [3]:
print("Beer Columns:")
print(df_beers.columns)
print("\nBrewery Columns:")
print(df_breweries.columns)
print("\nUsers Columns:")
print(df_users.columns)
print("\nRatings Columns:")
print(df_ratings_no_text.columns)

Beer Columns:
Index(['beer_id', 'beer_name', 'brewery_id', 'brewery_name', 'style', 'abv',
       'avg', 'std', 'median', 'appearance', 'aroma', 'palate', 'overall',
       'nbr_ratings', 'nbr_reviews', 'nbr_interactions'],
      dtype='object')

Brewery Columns:
Index(['id', 'location', 'name', 'nbr_beers'], dtype='object')

Users Columns:
Index(['user_id', 'user_name', 'location', 'joined', 'nbr_ratings',
       'nbr_reviews', 'nbr_interactions'],
      dtype='object')

Ratings Columns:
Index(['user_id', 'rating', 'review', 'abv', 'brewery_name', 'beer_id',
       'appearance', 'palate', 'aroma', 'overall', 'taste', 'style',
       'beer_name', 'brewery_id', 'date', 'idx', 'location', 'location_user',
       'location_brewery'],
      dtype='object')


In [4]:
# Select only countries/states with a significant number of ratings
MIN_RATINGS_THRESHOLD = 1
number_of_ratings_per_location = df_ratings_no_text['location_brewery'].value_counts()
popular_locations = number_of_ratings_per_location[number_of_ratings_per_location > MIN_RATINGS_THRESHOLD].index

# Filter the data
df_ratings_no_text = df_ratings_no_text[df_ratings_no_text['location_brewery'].isin(popular_locations)]
df_beers = df_beers[df_beers['beer_id'].isin(df_ratings_no_text['beer_id'].unique())]
df_breweries = df_breweries[df_breweries['id'].isin(df_ratings_no_text['brewery_id'].unique())]


In [ ]:
# Define a mapping of U.S. states to abbreviations
state_abbrev = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR",
    "California": "CA", "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE",
    "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID",
    "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS",
    "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD",
    "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN", "Mississippi": "MS",
    "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", "Nevada": "NV",
    "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM", "New York": "NY",
    "North Carolina": "NC", "North Dakota": "ND", "Ohio": "OH", "Oklahoma": "OK",
    "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC",
    "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT",
    "Vermont": "VT", "Virginia": "VA", "Washington": "WA", "West Virginia": "WV",
    "Wisconsin": "WI", "Wyoming": "WY"
}

# Add state and country information to the datasets
def add_location_info(df, location_column):
    df['is_us'] = df[location_column].str.contains("United States")
    df['state_name'] = df[location_column].where(df['is_us']).str.split(",").str[1]
    df['state_abbrev'] = df['state_name'].map(state_abbrev)
    df['country'] = df[location_column].where(~df['is_us'])
    return df

df_ratings_no_text = add_location_info(df_ratings_no_text, 'location_brewery')
df_breweries = add_location_info(df_breweries, 'location')
df_users = add_location_info(df_users, 'location')
df_beers = df_beers.merge(df_breweries[['id', 'location']], left_on='brewery_id', right_on='id')
df_beers = add_location_info(df_beers, 'location')


In [ ]:
# Ratings per country and state
ratings_per_country = df_ratings_no_text[~df_ratings_no_text['is_us']].groupby('country').size().reset_index(name='nbr_ratings')
ratings_per_state = df_ratings_no_text[df_ratings_no_text['is_us']].groupby('state_abbrev').size().reset_index(name='nbr_ratings')

# Breweries per country and state
breweries_per_country = df_breweries[~df_breweries['is_us']].groupby('country').size().reset_index(name='nbr_breweries')
breweries_per_state = df_breweries[df_breweries['is_us']].groupby('state_abbrev').size().reset_index(name='nbr_breweries')

# Users per country and state
users_per_country = df_users[~df_users['is_us']].groupby('country').size().reset_index(name='nbr_users')
users_per_state = df_users[df_users['is_us']].groupby('state_abbrev').size().reset_index(name='nbr_users')

# Beers per country and state
beers_per_country = df_beers[~df_beers['is_us']].groupby('country').size().reset_index(name='nbr_beers')
beers_per_state = df_beers[df_beers['is_us']].groupby('state_abbrev').size().reset_index(name='nbr_beers')


In [ ]:
import plotly.graph_objects as go

# Initialize figure
fig = go.Figure()

# Ratings
fig.add_trace(go.Choropleth(locations=ratings_per_country['country'], locationmode="country names",
                            z=ratings_per_country['nbr_ratings'], colorscale="Blues", colorbar_title="Ratings",
                            visible=True))
fig.add_trace(go.Choropleth(locations=ratings_per_state['state_abbrev'], locationmode="USA-states",
                            z=ratings_per_state['nbr_ratings'], colorscale="Blues", colorbar_title="Ratings",
                            visible=True))

# Breweries
fig.add_trace(go.Choropleth(locations=breweries_per_country['country'], locationmode="country names",
                            z=breweries_per_country['nbr_breweries'], colorscale="Reds", colorbar_title="Breweries",
                            visible=False))
fig.add_trace(go.Choropleth(locations=breweries_per_state['state_abbrev'], locationmode="USA-states",
                            z=breweries_per_state['nbr_breweries'], colorscale="Reds", colorbar_title="Breweries",
                            visible=False))

# Users
fig.add_trace(go.Choropleth(locations=users_per_country['country'], locationmode="country names",
                            z=users_per_country['nbr_users'], colorscale="Greens", colorbar_title="Users",
                            visible=False))
fig.add_trace(go.Choropleth(locations=users_per_state['state_abbrev'], locationmode="USA-states",
                            z=users_per_state['nbr_users'], colorscale="Greens", colorbar_title="Users",
                            visible=False))

# Beers
fig.add_trace(go.Choropleth(locations=beers_per_country['country'], locationmode="country names",
                            z=beers_per_country['nbr_beers'], colorscale="Oranges", colorbar_title="Beers",
                            visible=False))
fig.add_trace(go.Choropleth(locations=beers_per_state['state_abbrev'], locationmode="USA-states",
                            z=beers_per_state['nbr_beers'], colorscale="Oranges", colorbar_title="Beers",
                            visible=False))

# Dropdown menu
fig.update_layout(
    updatemenus=[
        dict(
            buttons=[
                dict(args=[{"visible": [True, True, False, False, False, False, False, False]}], label="Ratings", method="update"),
                dict(args=[{"visible": [False, False, True, True, False, False, False, False]}], label="Breweries", method="update"),
                dict(args=[{"visible": [False, False, False, False, True, True, False, False]}], label="Users", method="update"),
                dict(args=[{"visible": [False, False, False, False, False, False, True, True]}], label="Beers", method="update"),
            ],
            direction="down", x=0.1, y=1.15, xanchor="left", yanchor="top"
        )
    ],
    geo=dict(showframe=False, showcoastlines=True, projection_type="equirectangular"),
    height=600, width=800
)

# Show the plot
fig.show()

In [ ]:
# Initialize figure
fig_globe = go.Figure()

# Ratings
fig_globe.add_trace(go.Choropleth(locations=ratings_per_country['country'], locationmode="country names",
                                  z=ratings_per_country['nbr_ratings'], colorscale="Blues", colorbar_title="Ratings",
                                  visible=True))
fig_globe.add_trace(go.Choropleth(locations=ratings_per_state['state_abbrev'], locationmode="USA-states",
                                  z=ratings_per_state['nbr_ratings'], colorscale="Blues", colorbar_title="Ratings",
                                  visible=True))

# Breweries
fig_globe.add_trace(go.Choropleth(locations=breweries_per_country['country'], locationmode="country names",
                                  z=breweries_per_country['nbr_breweries'], colorscale="Reds", colorbar_title="Breweries",
                                  visible=False))
fig_globe.add_trace(go.Choropleth(locations=breweries_per_state['state_abbrev'], locationmode="USA-states",
                                  z=breweries_per_state['nbr_breweries'], colorscale="Reds", colorbar_title="Breweries",
                                  visible=False))

# Users
fig_globe.add_trace(go.Choropleth(locations=users_per_country['country'], locationmode="country names",
                                  z=users_per_country['nbr_users'], colorscale="Greens", colorbar_title="Users",
                                  visible=False))
fig_globe.add_trace(go.Choropleth(locations=users_per_state['state_abbrev'], locationmode="USA-states",
                                  z=users_per_state['nbr_users'], colorscale="Greens", colorbar_title="Users",
                                  visible=False))

# Beers
fig_globe.add_trace(go.Choropleth(locations=beers_per_country['country'], locationmode="country names",
                                  z=beers_per_country['nbr_beers'], colorscale="Oranges", colorbar_title="Beers",
                                  visible=False))
fig_globe.add_trace(go.Choropleth(locations=beers_per_state['state_abbrev'], locationmode="USA-states",
                                  z=beers_per_state['nbr_beers'], colorscale="Oranges", colorbar_title="Beers",
                                  visible=False))

# Dropdown menu
fig_globe.update_layout(
    updatemenus=[
        dict(
            buttons=[
                dict(args=[{"visible": [True, True, False, False, False, False, False, False]}], label="Ratings", method="update"),
                dict(args=[{"visible": [False, False, True, True, False, False, False, False]}], label="Breweries", method="update"),
                dict(args=[{"visible": [False, False, False, False, True, True, False, False]}], label="Users", method="update"),
                dict(args=[{"visible": [False, False, False, False, False, False, True, True]}], label="Beers", method="update"),
            ],
            direction="down", x=0.1, y=1.15, xanchor="left", yanchor="top"
        )
    ],
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type="orthographic",  # Use 'orthographic' for 3D globe view
        showcountries=True,
        showland=True,
        landcolor="rgb(217, 217, 217)",
        lakecolor="rgb(255, 255, 255)",
        oceancolor="rgb(204, 229, 255)",
        projection_rotation=dict(lat=10, lon=20),  # Starting position for the globe
    ),
    height=600,
    width=800
)

# Show the plot
fig_globe.show()
